# Setup

In [11]:
import pandas as pd
import numpy as np
from google.colab import files

In [12]:
#uploaded = files.upload()

In [13]:
import warnings
warnings.filterwarnings("ignore")
import io

# Load Data

In [14]:
df=pd.read_csv('experiment1.csv')
df['patterns'] = df.txt
#df['patterns'] = df.MsgBody

In [15]:
df.Celebrity.unique()

array(['BlackPink', 'Naeun Son', 'Kerwin Frost', 'Beyonce', 'Zoe Saldana',
       'Karlie Kloss', 'Yara Shahidi', 'Pharrell Williams',
       'Adriene Mishler', 'Ninjas Hyper', 'Bad Bunny', 'Jerry Lorenzo',
       'Chinae Alexander', 'Ally Love', 'Seolhyun', 'Solar', 'GFriend',
       'iZone', 'BTS', 'NCT'], dtype=object)

In [16]:
df.Celebrity.value_counts()

BlackPink            11507
Naeun Son             7751
NCT                   6348
Bad Bunny             5338
iZone                 4911
Ninjas Hyper          4742
Karlie Kloss          4587
GFriend               4492
Beyonce               3491
Yara Shahidi          3269
BTS                   3135
Zoe Saldana           2842
Kerwin Frost          2817
Seolhyun              2777
Solar                 2484
Pharrell Williams     2395
Jerry Lorenzo         2325
Chinae Alexander      1912
Ally Love             1776
Adriene Mishler        320
Name: Celebrity, dtype: int64

In [17]:
df.isnull().sum()

Unnamed: 0    0
Celebrity     0
author        0
txt           0
patterns      0
dtype: int64

# Clean data

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import string
from textblob import Word
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [19]:
#df=df.groupby(["Celebrity","txt"]).size().reset_index(name="freq")
#df['patterns'] = df.txt

In [20]:
stop = stopwords.words('english')
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x.lower() for x in x.split()))
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x for x in x.split() if x not in string.punctuation)) #remove punctuations
df['patterns'] = df['patterns'].str.replace('https*\S+','')  #remove url
df['patterns'] = df['patterns'].str.replace('\'\w+','')      #remove ticks
df['patterns'] = df['patterns'].str.replace('[^\w\s]','')    
df['patterns'] = df['patterns'].str.replace('@\S+','')       #remove email
df['patterns'] = df['patterns'].str.encode('ascii', 'ignore').str.decode("utf-8")  #remove unicode
df['patterns'] = df['patterns'].str.replace('\w*\d+\w*','')  #remove digits
df['patterns'] = df['patterns'].str.replace('#\S+','')  #remove hashtag
df['patterns'] = df['patterns'].str.replace('_','')  #remove underscore
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x for x in x.split() if not x in stop)) #remove stop words
df['patterns'] = df['patterns'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [21]:
col=["Celebrity","patterns"]
df_merge = df[col].groupby('Celebrity')['patterns'].apply(lambda x:x.str.cat(sep=" "))
df_merge

Celebrity
Adriene Mishler      hii im vicky let workout buddy stay motivated ...
Ally Love            would dismiss appropriation claim people take ...
BTS                  waititi love yes tf yall concerned fan upset f...
Bad Bunny            think alexa bliss nikki cross bigger feud rand...
Beyonce              beyonce new soda pop obnoxious way write note ...
BlackPink            deleted camera suck noticed since album dday v...
Chinae Alexander     thats compliment thats compliment thats compli...
GFriend              really hope swallow source music whole introdu...
Jerry Lorenzo        podcast uploaded come music fashion hard see k...
Karlie Kloss         event year love shit  linger like bad perfume ...
Kerwin Frost         people say design really mean innovation one t...
NCT                  loooooooove dream dont get wrong happy mark gr...
Naeun Son            bully compare irene scandal allegation true lo...
Ninjas Hyper           thoguht hyper came thouht glider already cam

In [22]:
for index, value in df_merge.items():
    print(f"Index : {index} \ {len(value.split())}")

Index : Adriene Mishler \ 2732
Index : Ally Love \ 22744
Index : BTS \ 49489
Index : Bad Bunny \ 63680
Index : Beyonce \ 34031
Index : BlackPink \ 125679
Index : Chinae Alexander \ 54177
Index : GFriend \ 58411
Index : Jerry Lorenzo \ 26858
Index : Karlie Kloss \ 43200
Index : Kerwin Frost \ 17288
Index : NCT \ 91198
Index : Naeun Son \ 115449
Index : Ninjas Hyper \ 67483
Index : Pharrell Williams \ 25046
Index : Seolhyun \ 34054
Index : Solar \ 37351
Index : Yara Shahidi \ 37215
Index : Zoe Saldana \ 26607
Index : iZone \ 72995


In [23]:
documents_df=pd.DataFrame(df_merge.values,columns=['documents'])
documents_df.index = df_merge.index

#### Define Cosine Similarity

In [24]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print (f'Celebrity: {documents_df.index[ix]}',":",similarity_matrix[doc_id][ix])
        #print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

# Tfidf with cosine similarity

https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [16]:
feature = 0
for i in documents_df.documents:
  feature += len(set(i.split()))

feature

76648

In [17]:
# removing special characters and stop words from the text
#stop_words_l=stopwords.words('english')
import math
max_feature = math.ceil(feature*0.7)
tfidfvectoriser=TfidfVectorizer(max_features=max_feature)
tfidfvectoriser.fit(documents_df.documents)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents)

In [18]:
tfidf_vectors.shape

(20, 27843)

In [19]:
tfidf_vectors=tfidf_vectors.toarray()

In [20]:
pairwise_similarities1=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences1=euclidean_distances(tfidf_vectors)

In [21]:
pairwise_similarities1.shape

(20, 20)

In [22]:
for i in range(len(df_merge)):
    print(i,":",df_merge.index[i])

0 : Adriene Mishler
1 : Ally Love
2 : BTS
3 : Bad Bunny
4 : Beyonce
5 : BlackPink
6 : Chinae Alexander
7 : GFriend
8 : Jerry Lorenzo
9 : Karlie Kloss
10 : Kerwin Frost
11 : NCT
12 : Naeun Son
13 : Ninjas Hyper
14 : Pharrell Williams
15 : Seolhyun
16 : Solar
17 : Yara Shahidi
18 : Zoe Saldana
19 : iZone


In [23]:
most_similar(5,pairwise_similarities1,'Cosine Similarity')

Document: documents    deleted camera suck noticed since album dday v...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: iZone : 0.6585091696081645
Celebrity: NCT : 0.6282038565075361
Celebrity: Solar : 0.5804038513815171
Celebrity: BTS : 0.5607910573543744
Celebrity: Seolhyun : 0.5354788082677442
Celebrity: GFriend : 0.5204510118404441
Celebrity: Beyonce : 0.5180485603999841
Celebrity: Naeun Son : 0.49397928944384156
Celebrity: Zoe Saldana : 0.4831891179665297
Celebrity: Jerry Lorenzo : 0.4718544073725158
Celebrity: Bad Bunny : 0.46981486076731527
Celebrity: Karlie Kloss : 0.4342190476220123
Celebrity: Yara Shahidi : 0.42702441943746233
Celebrity: Ninjas Hyper : 0.41961874403064786
Celebrity: Ally Love : 0.4191374744747143
Celebrity: Kerwin Frost : 0.36505256836967015
Celebrity: Pharrell Williams : 0.3379197423960877
Celebrity: Chinae Alexander : 0.274273120734555
Celebrity: Adriene Mishler : 0.11216821576928017


In [24]:
most_similar(5,pairwise_differences1,'Euclidean Distance')

Document: documents    deleted camera suck noticed since album dday v...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: iZone : 0.8264270450461386
Celebrity: NCT : 0.8623179732470735
Celebrity: Solar : 0.9160743950340386
Celebrity: BTS : 0.9372395026306125
Celebrity: Seolhyun : 0.9638684471775774
Celebrity: GFriend : 0.9793354769021322
Celebrity: Beyonce : 0.981785556626315
Celebrity: Naeun Son : 1.0060026943862184
Celebrity: Zoe Saldana : 1.0166719058117741
Celebrity: Jerry Lorenzo : 1.0277602761612126
Celebrity: Bad Bunny : 1.0297428215168025
Celebrity: Karlie Kloss : 1.0637489857837568
Celebrity: Yara Shahidi : 1.0704910840941562
Celebrity: Ninjas Hyper : 1.077386890554505
Celebrity: Ally Love : 1.0778334987606262
Celebrity: Kerwin Frost : 1.1268961191079954
Celebrity: Pharrell Williams : 1.1507217366539302
Celebrity: Chinae Alexander : 1.204762947027712
Celebrity: Adriene Mishler : 1.3325402689830588


# Word2vec + TFIDF with cosine similarity

In [25]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [26]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from gensim.models import Word2Vec

In [27]:
# tokenize and pad every document to make them of the same size
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=max_feature,padding='post')
vocab_size=len(tokenizer.word_index)+1

In [28]:
#### train word2vec
data=[]
for i in documents_df.documents:
    li = i.split()
    data.append(li)

This Google Developers blog post says:

Well, the following "formula" provides a general rule of thumb about the number of embedding dimensions:

embedding_dimensions =  number_of_categories**0.25

That is, the embedding vector dimension should be the 4th root of the number of categories.

Interestingly, the Word2vec Wikipedia article says (emphasis mine):

Nevertheless, for skip-gram models trained in medium size corpora, with 50 dimensions, a window size of 15 and 10 negative samples seems to be a good parameter setting.

Assuming a standard-ish sized vocabulary of 1.5 million words, this rule of thumb comes surprisingly close:

50 == 1.5e6 ** 0.2751

Parameters:
https://radimrehurek.com/gensim/models/word2vec.html

In [29]:
### calculate the vector_size
def flatten(data):
    return " ".join([str(item) for var in data for item in var])

num_words = 0
for item in data:
    num_words += len(item)
num_words

1005687

In [30]:
import math
dim_size = num_words**0.25
vector_size = math.ceil(float(dim_size))
vector_size

32

In [31]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
    def __init__(self, test_words):
        self.epoch = 1
        self._test_words = test_words
        self.loss_to_be_subed = 0

    def on_epoch_begin(self, model):
        print(f'Epoch: {self.epoch}', end='\t')

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print("Model loss:", loss_now)  # print loss
        #for word in self._test_words:  # show wv logic changes
        print(model.wv.most_similar(self._test_words,topn=1))
        self.epoch += 1

In [32]:
# Create skip-gram model
model = gensim.models.Word2Vec(data,
                                min_count = 1, 
                                vector_size = vector_size, 
                                window = 15, 
                                negative= 10,
                                sg=1,
                                compute_loss=True)
callback = MonitorCallback(["kpop"])
#model1.build_vocab(data)
model.train(data, total_examples=model.corpus_count,epochs=100,callbacks=[callback],compute_loss=True)

Epoch: 1	Model loss: 2003797.5
[('fansign', 0.8861509561538696)]
Epoch: 2	Model loss: 1790404.75
[('makestar', 0.9815424680709839)]
Epoch: 3	Model loss: 1777123.25
[('makestar', 1.0634592771530151)]
Epoch: 4	Model loss: 1711148.0
[('hwaiting', 2.275169610977173)]
Epoch: 5	Model loss: 1746051.5
[('hwaiting', 2.9441404342651367)]
Epoch: 6	Model loss: 1515450.0
[('brace', 5.81616735458374)]
Epoch: 7	Model loss: 1622889.0
[('brace', 6.06895637512207)]
Epoch: 8	Model loss: 1471115.0
[('brace', 6.184937477111816)]
Epoch: 9	Model loss: 1479876.0
[('maternity', 6.27163553237915)]
Epoch: 10	Model loss: 1552326.0
[('maternity', 6.903491973876953)]
Epoch: 11	Model loss: 1100807.0
[('maternity', 6.796426773071289)]
Epoch: 12	Model loss: 1385750.0
[('maternity', 6.737665176391602)]
Epoch: 13	Model loss: 1505770.0
[('maternity', 7.0605292320251465)]
Epoch: 14	Model loss: 1141378.0
[('maternity', 6.544722557067871)]
Epoch: 15	Model loss: 1309418.0
[('maternity', 7.229895114898682)]
Epoch: 16	Model lo

(19265293, 100568700)

In [47]:
model_w2v = model.wv

In [49]:
model.wv.most_similar("kpop",topn=10)

[('concept', 0.7486019730567932),
 ('rkpop', 0.7411696314811707),
 ('wjsn', 0.7324150204658508),
 ('bp', 0.7299093008041382),
 ('haha', 0.7265796065330505),
 ('comeback', 0.7214124798774719),
 ('sincd', 0.7199597358703613),
 ('beep', 0.717067301273346),
 ('boop', 0.7167270183563232),
 ('weekly', 0.7149010896682739)]

In [50]:
model.wv.most_similar("ninjashyper",topn=10)

[('ninjahyper', 0.9979957938194275),
 ('myth', 0.8917697072029114),
 ('forgetting', 0.7577356100082397),
 ('noscopes', 0.7355232238769531),
 ('creed', 0.7206867933273315),
 ('brotherhood', 0.7184299230575562),
 ('belief', 0.717257559299469),
 ('bastiat', 0.7083229422569275),
 ('hassan', 0.703590989112854),
 ('teets', 0.700646698474884)]

In [51]:
model.wv.most_similar("album",topn=10)

[('listened', 0.8081037402153015),
 ('invited', 0.7851409316062927),
 ('previously', 0.7755330204963684),
 ('pioneered', 0.7717604041099548),
 ('dynamite', 0.7670840620994568),
 ('jcole', 0.7665432691574097),
 ('surprising', 0.753007709980011),
 ('talked', 0.7487642168998718),
 ('listen', 0.7487235069274902),
 ('surprise', 0.7458373308181763)]

In [52]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
embedding_matrix=np.zeros((vocab_size,vector_size))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]
# creating document-word embeddings
document_word_embeddings=np.zeros((len(tokenized_paded_documents),max_feature,vector_size))
for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
document_word_embeddings.shape

(20, 53654, 32)

In [53]:
embedding_matrix[tokenizer.word_index['kpop']]

array([ 0.77277935, -0.06460083, -0.74197954, -0.19121847,  0.10651939,
       -0.72443002, -0.17206942, -0.14683576, -0.37938577,  0.34291658,
        0.0066812 , -0.84228891,  0.18180984, -0.32807183, -0.2433853 ,
        0.44291854,  0.74692839, -0.7239151 ,  0.88094664, -0.54813045,
        1.14617062, -0.41306737,  0.82859606,  0.03511152, -0.190953  ,
        0.12269177, -0.31739089, -0.03584532, -0.12187279, -0.68515253,
       -0.76915014, -0.08779287])

In [54]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents
document_embeddings=np.zeros((len(tokenized_paded_documents),vector_size))
words=tfidfvectoriser.get_feature_names()

In [55]:
for i in range(len(document_word_embeddings)):
  for j in range(len(words)):
      document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

In [56]:
pairwise_similarities2=cosine_similarity(document_embeddings)
pairwise_differences2=euclidean_distances(document_embeddings)

In [57]:
for i in range(len(df_merge)):
    print(i,":",df_merge.index[i])

0 : Adriene Mishler
1 : Ally Love
2 : BTS
3 : Bad Bunny
4 : Beyonce
5 : BlackPink
6 : Chinae Alexander
7 : GFriend
8 : Jerry Lorenzo
9 : Karlie Kloss
10 : Kerwin Frost
11 : NCT
12 : Naeun Son
13 : Ninjas Hyper
14 : Pharrell Williams
15 : Seolhyun
16 : Solar
17 : Yara Shahidi
18 : Zoe Saldana
19 : iZone


In [58]:
most_similar(5,pairwise_similarities2,'Cosine Similarity') #most_similar(5,pairwise_differences2,'Euclidean Distance')

Document: documents    deleted camera suck noticed since album dday v...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: NCT : 0.988126639726233
Celebrity: iZone : 0.9840933356716458
Celebrity: Solar : 0.9804480263533095
Celebrity: BTS : 0.9773551860971278
Celebrity: Naeun Son : 0.9740182190302626
Celebrity: GFriend : 0.9739962507584092
Celebrity: Beyonce : 0.9609475107502808
Celebrity: Seolhyun : 0.9602238213055705
Celebrity: Ally Love : 0.9565573191060359
Celebrity: Zoe Saldana : 0.9531724326140566
Celebrity: Jerry Lorenzo : 0.9501476145414024
Celebrity: Bad Bunny : 0.9454061659610573
Celebrity: Yara Shahidi : 0.9451414773614599
Celebrity: Karlie Kloss : 0.935789520086869
Celebrity: Kerwin Frost : 0.9302353442112984
Celebrity: Pharrell Williams : 0.9241715625651554
Celebrity: Ninjas Hyper : 0.9223248697237673
Celebrity: Chinae Alexander : 0.8732276591029622
Celebrity: Adriene Mishler : 0.8500278609500979


In [59]:
most_similar(13,pairwise_similarities2,'Cosine Similarity')

Document: documents      thoguht hyper came thouht glider already cam...
Name: Ninjas Hyper, dtype: object


Similar Documents:
Celebrity: Ally Love : 0.9518855926375056
Celebrity: Jerry Lorenzo : 0.9451542079055322
Celebrity: Yara Shahidi : 0.9451341225941053
Celebrity: Chinae Alexander : 0.9380682938402194
Celebrity: Bad Bunny : 0.9355682399501649
Celebrity: NCT : 0.9235316945341204
Celebrity: BlackPink : 0.9223248697237673
Celebrity: Kerwin Frost : 0.9211978317047808
Celebrity: Beyonce : 0.9208059597295435
Celebrity: Solar : 0.9185731785248524
Celebrity: Zoe Saldana : 0.9147846327617434
Celebrity: iZone : 0.9147825289236904
Celebrity: BTS : 0.910046608996171
Celebrity: GFriend : 0.9088317774438871
Celebrity: Naeun Son : 0.8939816599778141
Celebrity: Karlie Kloss : 0.8841421667899639
Celebrity: Pharrell Williams : 0.8817826552583412
Celebrity: Seolhyun : 0.8788907132075058
Celebrity: Adriene Mishler : 0.7729268880658977


In [60]:
most_similar(4,pairwise_similarities2,'Cosine Similarity') #most_similar(5,pairwise_differences2,'Euclidean Distance')

Document: documents    beyonce new soda pop obnoxious way write note ...
Name: Beyonce, dtype: object


Similar Documents:
Celebrity: Zoe Saldana : 0.9990190226591077
Celebrity: BlackPink : 0.9609475107502808
Celebrity: NCT : 0.954587870101293
Celebrity: Bad Bunny : 0.9540561533506211
Celebrity: Pharrell Williams : 0.9538972403082145
Celebrity: BTS : 0.9525568632245527
Celebrity: Solar : 0.9521816239838274
Celebrity: Naeun Son : 0.9509633106219129
Celebrity: Yara Shahidi : 0.9465443266807936
Celebrity: Karlie Kloss : 0.9440205106484143
Celebrity: iZone : 0.9437049063248565
Celebrity: Jerry Lorenzo : 0.9433354645067069
Celebrity: Kerwin Frost : 0.9394107391977032
Celebrity: GFriend : 0.9389407237886923
Celebrity: Ally Love : 0.929961434787811
Celebrity: Seolhyun : 0.9210656741646683
Celebrity: Ninjas Hyper : 0.9208059597295435
Celebrity: Chinae Alexander : 0.8814955795446007
Celebrity: Adriene Mishler : 0.7957443632486244


In [61]:
most_similar(12,pairwise_similarities2,'Cosine Similarity') #most_similar(5,pairwise_differences2,'Euclidean Distance')

Document: documents    bully compare irene scandal allegation true lo...
Name: Naeun Son, dtype: object


Similar Documents:
Celebrity: iZone : 0.9892696485246559
Celebrity: Solar : 0.9870124225141712
Celebrity: NCT : 0.9842096528674684
Celebrity: Seolhyun : 0.98327417256649
Celebrity: BTS : 0.9827147710362689
Celebrity: GFriend : 0.9780483888510311
Celebrity: BlackPink : 0.9740182190302626
Celebrity: Beyonce : 0.9509633106219129
Celebrity: Ally Love : 0.9451126088249752
Celebrity: Zoe Saldana : 0.9449952009598405
Celebrity: Bad Bunny : 0.9394753057895053
Celebrity: Yara Shahidi : 0.9296842711458398
Celebrity: Jerry Lorenzo : 0.929391528023178
Celebrity: Karlie Kloss : 0.9121899426963066
Celebrity: Kerwin Frost : 0.901364086514188
Celebrity: Pharrell Williams : 0.894199228815855
Celebrity: Ninjas Hyper : 0.8939816599778141
Celebrity: Chinae Alexander : 0.8789729270476289
Celebrity: Adriene Mishler : 0.828078912234279


# Tfidf + GloVe with cosine similarity

Get the pre-trained GloVe model from Stanford, at: https://figshare.com/articles/dataset/Twitter_pre-trained_word_vectors/11640300

In [151]:
# reading Glove word embeddings into a dictionary with "word" as key and values as word vectors

embeddings_index = dict()

with open("glove.twitter.27B.200d.txt") as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [152]:
len(embeddings_index)

980499

In [153]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 

embedding_matrix=np.zeros((vocab_size,len(embeddings_index["key"])))

for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [154]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),len(embeddings_index["key"])))
words=tfidfvectoriser.get_feature_names()

for i in range(documents_df.shape[0]):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

In [155]:
document_embeddings.shape

(20, 200)

In [156]:
pairwise_similarities3=cosine_similarity(document_embeddings)
pairwise_differences3=euclidean_distances(document_embeddings)

In [157]:
most_similar(5,pairwise_similarities3,'Cosine Similarity')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: iZone : 0.9986147186201472
Celebrity: BTS : 0.9983209532295985
Celebrity: GFriend : 0.9982327167351418
Celebrity: Solar : 0.9981660576630863
Celebrity: NCT : 0.9980128366889742
Celebrity: Seolhyun : 0.9963437236349071
Celebrity: Naeun Son : 0.9945692421547004
Celebrity: Yara Shahidi : 0.9934696933776209
Celebrity: Ally Love : 0.993342570324519
Celebrity: Karlie Kloss : 0.9927576865787932
Celebrity: Zoe Saldana : 0.9918334900261311
Celebrity: Beyonce : 0.991796426695931
Celebrity: Bad Bunny : 0.990911168873496
Celebrity: Kerwin Frost : 0.9896617102851425
Celebrity: Jerry Lorenzo : 0.9886843404633153
Celebrity: Pharrell Williams : 0.9882151526271628
Celebrity: Ninjas Hyper : 0.9862473770232395
Celebrity: Chinae Alexander : 0.975340574283
Celebrity: Adriene Mishler : 0.9709562089288194


In [158]:
most_similar(5,pairwise_differences3,'Euclidean Distance')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: BTS : 0.22204474943420463
Celebrity: iZone : 0.24297856543285398
Celebrity: Solar : 0.2481551325366072
Celebrity: GFriend : 0.32833901579826597
Celebrity: Seolhyun : 0.33258690928254375
Celebrity: NCT : 0.3444528043067681
Celebrity: Naeun Son : 0.42734693277066327
Celebrity: Karlie Kloss : 0.46089980951574056
Celebrity: Yara Shahidi : 0.4706895403466627
Celebrity: Zoe Saldana : 0.48987314908305296
Celebrity: Beyonce : 0.49094123876218976
Celebrity: Bad Bunny : 0.519789636872434
Celebrity: Ally Love : 0.5574818882642636
Celebrity: Jerry Lorenzo : 0.581787054633393
Celebrity: Pharrell Williams : 0.5880021849336043
Celebrity: Kerwin Frost : 0.6219333577007011
Celebrity: Ninjas Hyper : 0.6342943485838276
Celebrity: Chinae Alexander : 0.8664561165334612
Celebrity: Adriene Mishler : 1.1600514720747888


# Doc2vec with cosine similarity




In [120]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [154]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[index]) for index, doc in enumerate(documents_df.documents)]

In [163]:
model_d2v = Doc2Vec(vector_size=200,alpha=0.025, 
                    min_count=1,
                    window=15, 
                    negative=10, 
                    dm_mean=1,
                    dbow_words=1
                    )
  
model_d2v.build_vocab(tagged_data)

for epoch in range(200):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

In [164]:
document_embeddings=np.zeros((documents_df.shape[0],200))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

In [165]:
pairwise_similarities4=cosine_similarity(document_embeddings)
pairwise_differences4=euclidean_distances(document_embeddings)

In [166]:
most_similar(5,pairwise_similarities4,'Cosine Similarity')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: Naeun Son : 0.9532936044514828
Celebrity: BTS : 0.9527094450557538
Celebrity: GFriend : 0.9496424667579314
Celebrity: Solar : 0.9490676454970279
Celebrity: Beyonce : 0.9489000390054098
Celebrity: iZone : 0.9471757897682429
Celebrity: NCT : 0.946129480248482
Celebrity: Bad Bunny : 0.9454458585754192
Celebrity: Zoe Saldana : 0.9453162151296085
Celebrity: Chinae Alexander : 0.9440401597176502
Celebrity: Ally Love : 0.9411788655820581
Celebrity: Jerry Lorenzo : 0.940393321210593
Celebrity: Ninjas Hyper : 0.9368134333631974
Celebrity: Seolhyun : 0.9359839708528181
Celebrity: Karlie Kloss : 0.9280650087730549
Celebrity: Pharrell Williams : 0.9274005255266431
Celebrity: Kerwin Frost : 0.9170355502833284
Celebrity: Yara Shahidi : 0.9022188446436693
Celebrity: Adriene Mishler : 0.8660213846534885


In [167]:
most_similar(5,pairwise_differences4,'Euclidean Distance')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: Naeun Son : 29.73081153144812
Celebrity: BTS : 30.16684669021789
Celebrity: GFriend : 30.898473157575935
Celebrity: Solar : 31.12984106151588
Celebrity: iZone : 31.772186034807284
Celebrity: NCT : 32.113347746757725
Celebrity: Bad Bunny : 33.63302798847993
Celebrity: Beyonce : 34.38650994456916
Celebrity: Seolhyun : 35.76458960487477
Celebrity: Zoe Saldana : 35.85441841989864
Celebrity: Ally Love : 36.31135418444681
Celebrity: Chinae Alexander : 36.471290116292224
Celebrity: Jerry Lorenzo : 37.07216804082898
Celebrity: Ninjas Hyper : 39.41940215094015
Celebrity: Pharrell Williams : 40.32330367023687
Celebrity: Karlie Kloss : 40.37005096249041
Celebrity: Kerwin Frost : 42.8547997704036
Celebrity: Yara Shahidi : 48.46079103946066
Celebrity: Adriene Mishler : 50.904733370976814


# BERT model for sentence embedding

In [45]:
from sentence_transformers import SentenceTransformer

Pre-trained model: https://www.sbert.net/docs/pretrained_models.html

##### Paraphrase Identification
The following models are recommended for various applications, as they were trained on Millions of paraphrase examples. They create extremely good results for various similarity and retrieval tasks. They are currently under development, better versions and more details will be released in future. But they many tasks they work better than the NLI / STSb models.

**paraphrase-distilroberta-base-v1 - Trained on large scale paraphrase data.**

**paraphrase-xlm-r-multilingual-v1 - Multilingual version of** 

**paraphrase-distilroberta-base-v1, trained on parallel data for 50+ languages.** 

(Teacher: paraphrase-distilroberta-base-v1, Student: xlm-r-base)

##### Semantic Textual Similarity
The following models were optimized for Semantic Textual Similarity (STS). They were trained on SNLI+MultiNLI and then fine-tuned on the STS benchmark train set.

The best available models for STS are:

**stsb-roberta-large - STSb performance: 86.39**

**stsb-roberta-base - STSb performance: 85.44**

**stsb-bert-large - STSb performance: 85.29**

**stsb-distilbert-base - STSb performance: 85.16**

» Full List of STS Models

##### Duplicate Questions Detection
The following models were trained for duplicate questions mining and duplicate questions retrieval. You can use them to detect duplicate questions in a large corpus (see paraphrase mining) or to search for similar questions (see semantic search).

Available models:

**quora-distilbert-base - Model first tuned on NLI+STSb data, then fine-tune* for Quora Duplicate Questions detection retrieval.**

**quora-distilbert-multilingual - Multilingual version of quora-distilbert-base. Fine-tuned with parallel data for 50+ languages.**

##### Question-Answer Retrieval - MSMARCO
The following models were trained on MSMARCO Passage Ranking, a dataset with 500k real queries from Bing search. Given a search query, find the relevant passages.

**msmarco-distilbert-base-v3: MRR@10: 33.13 on MS MARCO dev set**

**msmarco-roberta-base-ance-fristp: MRR@10: 33.03 on MS MARCO dev set**

In [192]:
sbert_model = SentenceTransformer('stsb-bert-large')

In [193]:
document_embeddings = sbert_model.encode(documents_df['documents'])

In [194]:
len(document_embeddings)

20

In [195]:
pairwise_similarities5=cosine_similarity(document_embeddings)
pairwise_differences5=euclidean_distances(document_embeddings)

In [196]:
for i in range(len(df_merge.index)):
  print(i,":",df_merge.index[i])

0 : Adriene Mishler
1 : Ally Love
2 : BTS
3 : Bad Bunny
4 : Beyonce
5 : BlackPink
6 : Chinae Alexander
7 : GFriend
8 : Jerry Lorenzo
9 : Karlie Kloss
10 : Kerwin Frost
11 : NCT
12 : Naeun Son
13 : Ninjas Hyper
14 : Pharrell Williams
15 : Seolhyun
16 : Solar
17 : Yara Shahidi
18 : Zoe Saldana
19 : iZone


In [197]:
most_similar(5,pairwise_similarities5,'Cosine Similarity')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: Pharrell Williams : 0.7075412
Celebrity: Karlie Kloss : 0.68185306
Celebrity: Zoe Saldana : 0.65860915
Celebrity: Beyonce : 0.65860915
Celebrity: Yara Shahidi : 0.625318
Celebrity: Ally Love : 0.62436926
Celebrity: GFriend : 0.56523216
Celebrity: Adriene Mishler : 0.5646051
Celebrity: NCT : 0.5564555
Celebrity: BTS : 0.51107275
Celebrity: Bad Bunny : 0.50973934
Celebrity: Solar : 0.49988914
Celebrity: Jerry Lorenzo : 0.48019633
Celebrity: Ninjas Hyper : 0.47468945
Celebrity: Kerwin Frost : 0.44876078
Celebrity: iZone : 0.4053278
Celebrity: Naeun Son : 0.36872032
Celebrity: Seolhyun : 0.36180216
Celebrity: Chinae Alexander : 0.3482866


In [191]:
most_similar(5,pairwise_differences5,'Euclidean Distance')

Document: documents    cute someone fix neck hey thinker great post t...
Name: BlackPink, dtype: object


Similar Documents:
Celebrity: Pharrell Williams : 10.804895
Celebrity: Zoe Saldana : 11.465481
Celebrity: Beyonce : 11.465481
Celebrity: Yara Shahidi : 12.098975
Celebrity: Adriene Mishler : 13.202191
Celebrity: Karlie Kloss : 13.329329
Celebrity: Jerry Lorenzo : 13.446043
Celebrity: Ally Love : 13.447281
Celebrity: Bad Bunny : 14.777736
Celebrity: NCT : 14.902348
Celebrity: Chinae Alexander : 15.251679
Celebrity: Solar : 15.393733
Celebrity: Ninjas Hyper : 15.755608
Celebrity: BTS : 15.821575
Celebrity: Seolhyun : 16.018211
Celebrity: GFriend : 16.039948
Celebrity: iZone : 16.212719
Celebrity: Kerwin Frost : 16.625265
Celebrity: Naeun Son : 18.085722


# Sent2vec

In [25]:
pip install sent2vec

     |████████████████████████████████| 2.0MB 4.9MB/s 
     |████████████████████████████████| 3.2MB 22.6MB/s 
     |████████████████████████████████| 870kB 36.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=7d250715e897b4eea85148369bdac29562cce62fd3ca8538adde7525da93e004
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [26]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

In [42]:
string = []
for i in documents_df.documents.to_list():
  string.append(str(i))

print(len(string))

20


In [ ]:
sentences = str(documents_df.documents.to_list())

vectorizer = Vectorizer()
vectorizer.bert(sentences)
vectors_bert = vectorizer.vectors

In [ ]:
dist_1 = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])
dist_2 = spatial.distance.cosine(vectors_bert[0], vectors_bert[2])
print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))
# dist_1: 0.043, dist_2: 0.192